**Actual Machine Learning**

In [ ]:
# tqdm
from pathlib import Path
import Augmentation
import torchaudio
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import datasets
import numpy as np
from torch.utils.data import Dataset, DataLoader
from datetime import datetime
import os
import time

import mlmodel

audio_paths = Augmentation.getAudioPaths('./UrbanSounds8K/')[0:10]

audio_train_paths, audio_val_paths = audio_paths[:int(
    0.8 * len(audio_paths))], audio_paths[int(0.8 * len(audio_paths)):]

audio_train_dataset = Augmentation.AudioDataset(
    audio_train_paths,
    transformList=[
        torchaudio.transforms.TimeMasking(time_mask_param=80),
        torchaudio.transforms.FrequencyMasking(freq_mask_param=80),
    ])

audio_val_dataset = Augmentation.AudioDataset(audio_val_paths)

train_dataloader = torch.utils.data.DataLoader(audio_train_dataset,
                                               batch_size=4,
                                               num_workers=0,
                                               shuffle=True)

val_dataloader = torch.utils.data.DataLoader(audio_val_dataset,
                                              batch_size=4,
                                              num_workers=0,
                                              shuffle=True,
                                             )

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18')

model.conv1 = nn.Conv2d(1,
                        64,
                        kernel_size=(7, 7),
                        stride=(2, 2),
                        padding=(3, 3),
                        bias=False)

model.fc = nn.Linear(in_features=512, out_features=10, bias=True)
cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

title = datetime.now().strftime("%Y-%m-%d,%H-%M-%S")
title = False
mymodel = mlmodel.mlmodel(model, optimizer, train_dataloader, val_dataloader,
                          torch.nn.CrossEntropyLoss(), title)

In [ ]:
# mymodel.write_tb_graph(train_dataloader)

In [ ]:
mymodel.interateModel(epochs = 100)

In [ ]:
model.load_state_dict(
    torch.load("./model/model_t4,f4,tf4.pt", map_location=device))

In [ ]:
audio_test_dataset = datasets.DatasetFolder(root='./UrbanSounds8K/spectrograms/test/',
                                       loader=npy_loader,
                                       extensions=['.npy'])

test_dataloader = torch.utils.data.DataLoader(audio_test_dataset,
                                              batch_size=16,
                                              num_workers=0,
                                              shuffle=True,
                                             )

val(val_dataloader, model)

In [ ]:
torch.save(model.state_dict(), f"./model/model_t4,f4,tf4.pt")

**Debugging**

In [ ]:

# audio_dataset = []
# sum = 0

# main_path = './UrbanSounds8K/spectrograms/'

# dir = [str(p) for p in Path(main_path).glob('*')]
# for path in dir:
#   print('Loading: '+ path)
#   num = len([str(p) for p in Path(path).glob('*')])
#   sum += num
#   print(num)

# sum